In [1]:
import tensorflow as tf
import numpy as np

* 學習於[https://tf.wiki/zh_hans/basic/models.html](https://tf.wiki/zh_hans/basic/models.html)
## Multilayer Perceptron(MLP)
* 又可稱多層全連接神經網路
* 步驟
    1. 使用 **tf.keras.dataset** 獲得dataset並預處理
    2. 使用 **tf.keras.Model** 和 **tf.keras,layers** 建構模型
    3. 使用 **tf.keras.losses** 計算loss function
    4. 使用 **tf.keras.optimizer** 優化模型
    5. 使用 **tf.keras.metrics** 計算評估指標

### dataset與pre-process
* 做一個簡單的 MNISTLoader class 來讀取 MNIST dataset
<img src="https://tf.wiki/_images/mnist_0-9.png">
* 分別載入60000和10000張大小為28x28的手寫數字圖片

In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # train_data.shape = (60000, 28, 28): 60000個28x28的input
        # 將像素都坐正規化，並且增加一個維度存放channel數量(1,因為為灰階)
        # 用astype()轉型
        self.train_data = np.expand_dims(self.train_data.astype(np.float32)/255.0, axis=-1) #[60000,28,28,1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32)/255.0, axis=-1) #[10000,28,28,1]
        self.train_label = self.train_label.astype(np.int32) #[60000]
        self.test_label = self.test_label.astype(np.int32) #[10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
        
    def get_batch(self, batch_size):
        # 從資料集中隨機存取batch_size個元素return
        # index為存放的要挑選的index(一維陣列)
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

### 模型構建
* 使用 **tf.keras.Model** 和 **tf.keras.layers**
* 該model輸入為一個向量(拉直的1x784)手寫數字圖片
* 輸出10維的向量，分別代表0到9的機率

#### Softmax
* 我們希望輸出為每個 output label 的機率 => 一個10維離散機率
    * 該向量中的每個元素均在 **[0,1]** 之間
    * 該向量的所有元素之和為 **1**
    <img src="img/softmax.png">

In [3]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # Flatten層將除了第一維(batch_size)以外的維度攤平
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
    
    def call(self, inputs):      # [batch_size, 28, 28, 1]
        x = self.flatten(inputs) # [batch_size, 784]
        x = self.dense1(x)       # [batch_size, 100]
        x = self.dense2(x)       # [batch_size, 8]
        output = tf.nn.softmax(x)
        return output

<img src="https://tf.wiki/_images/mlp.png">

### 模型的訓練 

In [4]:
# 定義一些模型的 hyper paremeter
num_epochs = 5
batch_size = 50
learning_rate = 0.001

### 優化器選擇
參考[網站](https://medium.com/雞雞與兔兔的工程世界/機器學習ml-note-sgd-momentum-adagrad-adam-optimizer-f20568c968db)
* SGD-準確率梯度下降法 (stochastic gradient decent)
    * <img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*C8PAvTAXukHZ2mZPItwIrg.png">
* Momentum
    * 此優化器為模擬物理動量的概念，在同方向的維度上學習速度會變快，方向改變的時候學習速度會變慢
    * <img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*tbRXwsHwo9WPx5xTcZDX_A.png">
    * **Vt** 可以想像成 **方向速度**，會與上一次的更新有關，如果上一次的梯度與這次同方向的話，|Vt|(速度)會越來越大(表示梯度增強)，w參數的更新梯度便會越來越快
    * **$\beta$** 可以想像為空氣阻力或是地面摩擦力，通常設為0.9
* AdaGrad(Adaptive)
    * learning rate η 對優化器非常重要，太小會花費太多時間學習，太大可能會overfitting，無法正確學習
    * AdaGrad會依照梯度去調整 η
    * <img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*203QdRb0FaCFNfIZayfUsw.png">
        * 其中 η 乘上 1/√(n+ϵ) 再做參數更新
        * n為前面所有梯度值得平方和
        * ϵ 為平滑值，加上 ϵ 的原因是為了不讓分母為0，ϵ 一般值為1e-8
    * 特性
        1. 前期梯度較小的時候，n較小，能夠放大學習率
        2. 後期梯度較大的時候，n較大，能夠約束學習率
* Adam
    * Momentum 跟 AdaGrad這二種Optimizer做結合，為目前較常使用的Optimizer
    * <img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*HIrsnzAhkYsm1wCI6gUaug.png">
        * 像Momentum一樣保持了過去梯度的指數衰減平均值，像AdaGrad一樣存了過去梯度的平方衰減平均值
    * <img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*4b7GUhHlzoqum1U4PCTFWw.png">
        * 對mt跟vt做偏離校正
    * <img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*uhyuvY_h-NFYU_hmKwfaFQ.png">
        * Adam 保留了 Momentum 對過去梯度的方向做梯度速度調整與AdaGrad對過去梯度的平方值做learning rate的調整，再加上Adam有做參數的”偏離校正”，使得每一次的學習率都會有個確定的範圍，會讓參數的更新較為平穩 
        
### 總整理[參考網站](https://medium.com/雞雞與兔兔的工程世界/機器學習ml-note-sgd-momentum-adagrad-adam-optimizer-f20568c968db)
<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*U0IT4yJReyPibVI43j38cQ.png">
<img src="https://miro.medium.com/v2/resize:fit:1240/1*SjtKOauOXFVjWRR7iCtHiA.gif">

In [5]:
model = MLP()
data_loader = MNISTLoader()
# 選擇Adam作為優化器
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

Metal device set to: Apple M1


2023-03-26 15:21:29.498420: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-26 15:21:29.498783: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
# 從data loader中隨機取一批訓練資料
# num_batches為batches的數量
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
# Epoch這是指當所有資料都被用來訓練類神經網路一次
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print(f"batch {batch_index}: loss {loss.numpy()}")
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.3370795249938965
batch 1: loss 2.287160873413086
batch 2: loss 2.181264638900757
batch 3: loss 2.0944173336029053
batch 4: loss 1.989709734916687
batch 5: loss 1.9161925315856934
batch 6: loss 1.966715931892395
batch 7: loss 1.8699395656585693
batch 8: loss 1.7662453651428223
batch 9: loss 1.6430126428604126
batch 10: loss 1.5790722370147705
batch 11: loss 1.697829246520996
batch 12: loss 1.4866368770599365
batch 13: loss 1.37369966506958
batch 14: loss 1.3470656871795654
batch 15: loss 1.2022373676300049
batch 16: loss 1.3854176998138428
batch 17: loss 1.2590304613113403
batch 18: loss 1.443745493888855
batch 19: loss 1.2901467084884644
batch 20: loss 1.1727484464645386
batch 21: loss 1.3447316884994507
batch 22: loss 1.1882752180099487
batch 23: loss 1.1394003629684448
batch 24: loss 0.9043923020362854
batch 25: loss 1.1118465662002563
batch 26: loss 1.2136934995651245
batch 27: loss 0.8705396056175232
batch 28: loss 0.8899882435798645
batch 29: loss 0.970076441764831

batch 252: loss 0.21521815657615662
batch 253: loss 0.3988133668899536
batch 254: loss 0.21556247770786285
batch 255: loss 0.2677079141139984
batch 256: loss 0.34553390741348267
batch 257: loss 0.16768860816955566
batch 258: loss 0.5390146374702454
batch 259: loss 0.26769259572029114
batch 260: loss 0.3008610010147095
batch 261: loss 0.21756117045879364
batch 262: loss 0.452384889125824
batch 263: loss 0.3166040778160095
batch 264: loss 0.14949947595596313
batch 265: loss 0.3611223101615906
batch 266: loss 0.29933077096939087
batch 267: loss 0.19630639255046844
batch 268: loss 0.5219059586524963
batch 269: loss 0.49085530638694763
batch 270: loss 0.21292737126350403
batch 271: loss 0.30116796493530273
batch 272: loss 0.35086944699287415
batch 273: loss 0.2902015149593353
batch 274: loss 0.2660571038722992
batch 275: loss 0.23285678029060364
batch 276: loss 0.4210340678691864
batch 277: loss 0.28089237213134766
batch 278: loss 0.2126895636320114
batch 279: loss 0.2748216688632965
batch 

batch 499: loss 0.20532234013080597
batch 500: loss 0.7155076265335083
batch 501: loss 0.16050133109092712
batch 502: loss 0.10372867435216904
batch 503: loss 0.4076671600341797
batch 504: loss 0.09948620945215225
batch 505: loss 0.4464510679244995
batch 506: loss 0.17921182513237
batch 507: loss 0.14739641547203064
batch 508: loss 0.29852885007858276
batch 509: loss 0.4577837884426117
batch 510: loss 0.21631889045238495
batch 511: loss 0.3443073332309723
batch 512: loss 0.2657000422477722
batch 513: loss 0.17890074849128723
batch 514: loss 0.2537586987018585
batch 515: loss 0.1849393993616104
batch 516: loss 0.3671322464942932
batch 517: loss 0.3183537721633911
batch 518: loss 0.19231656193733215
batch 519: loss 0.26052072644233704
batch 520: loss 0.2900692820549011
batch 521: loss 0.2156733125448227
batch 522: loss 0.18374092876911163
batch 523: loss 0.31591278314590454
batch 524: loss 0.28520914912223816
batch 525: loss 0.16423122584819794
batch 526: loss 0.38088446855545044
batch 5

batch 758: loss 0.2842966616153717
batch 759: loss 0.20970191061496735
batch 760: loss 0.32217949628829956
batch 761: loss 0.15411075949668884
batch 762: loss 0.102567158639431
batch 763: loss 0.23558032512664795
batch 764: loss 0.10985280573368073
batch 765: loss 0.21874557435512543
batch 766: loss 0.11989489942789078
batch 767: loss 0.4594474732875824
batch 768: loss 0.22298561036586761
batch 769: loss 0.16461536288261414
batch 770: loss 0.2969748377799988
batch 771: loss 0.22994393110275269
batch 772: loss 0.0861022099852562
batch 773: loss 0.19118236005306244
batch 774: loss 0.11846795678138733
batch 775: loss 0.3656146824359894
batch 776: loss 0.1644221395254135
batch 777: loss 0.2079099863767624
batch 778: loss 0.07584474980831146
batch 779: loss 0.263822466135025
batch 780: loss 0.16324229538440704
batch 781: loss 0.03529270738363266
batch 782: loss 0.242379829287529
batch 783: loss 0.2420186996459961
batch 784: loss 0.11078204959630966
batch 785: loss 0.13609179854393005
batch 

batch 1013: loss 0.09135940670967102
batch 1014: loss 0.3697722554206848
batch 1015: loss 0.2732320725917816
batch 1016: loss 0.12711597979068756
batch 1017: loss 0.1084197610616684
batch 1018: loss 0.09602268040180206
batch 1019: loss 0.17100539803504944
batch 1020: loss 0.35996997356414795
batch 1021: loss 0.17367111146450043
batch 1022: loss 0.18135829269886017
batch 1023: loss 0.20467212796211243
batch 1024: loss 0.33018797636032104
batch 1025: loss 0.24753817915916443
batch 1026: loss 0.14725607633590698
batch 1027: loss 0.15417304635047913
batch 1028: loss 0.15562553703784943
batch 1029: loss 0.09998609125614166
batch 1030: loss 0.2256435751914978
batch 1031: loss 0.14071819186210632
batch 1032: loss 0.1354774683713913
batch 1033: loss 0.06417471915483475
batch 1034: loss 0.12619896233081818
batch 1035: loss 0.0912032276391983
batch 1036: loss 0.14519207179546356
batch 1037: loss 0.1051141545176506
batch 1038: loss 0.14408215880393982
batch 1039: loss 0.20723624527454376
batch 10

...

batch 4919: loss 0.052892979234457016
batch 4920: loss 0.0937715619802475
batch 4921: loss 0.06274821609258652
batch 4922: loss 0.044390905648469925
batch 4923: loss 0.12508781254291534
batch 4924: loss 0.09197679162025452
batch 4925: loss 0.01023838296532631
batch 4926: loss 0.04252065718173981
batch 4927: loss 0.009032294154167175
batch 4928: loss 0.06254056841135025
batch 4929: loss 0.13622261583805084
batch 4930: loss 0.012982625514268875
batch 4931: loss 0.01126958429813385
batch 4932: loss 0.2361791580915451
batch 4933: loss 0.04636678472161293
batch 4934: loss 0.061300162225961685
batch 4935: loss 0.04163951426744461
batch 4936: loss 0.08000452071428299
batch 4937: loss 0.10513977706432343
batch 4938: loss 0.07086271792650223
batch 4939: loss 0.10618475824594498
batch 4940: loss 0.10804872959852219
batch 4941: loss 0.022961871698498726
batch 4942: loss 0.06637685745954514
batch 4943: loss 0.042823467403650284
batch 4944: loss 0.04062414914369583
batch 4945: loss 0.04805713519454

batch 5152: loss 0.023102696985006332
batch 5153: loss 0.04150742292404175
batch 5154: loss 0.12350206822156906
batch 5155: loss 0.038473162800073624
batch 5156: loss 0.2400617152452469
batch 5157: loss 0.05360627546906471
batch 5158: loss 0.012905625626444817
batch 5159: loss 0.03011118434369564
batch 5160: loss 0.12793321907520294
batch 5161: loss 0.06560973823070526
batch 5162: loss 0.07661076635122299
batch 5163: loss 0.03989051282405853
batch 5164: loss 0.009472118690609932
batch 5165: loss 0.07948201149702072
batch 5166: loss 0.014187612570822239
batch 5167: loss 0.008983428589999676
batch 5168: loss 0.0655910074710846
batch 5169: loss 0.06710302084684372
batch 5170: loss 0.20707343518733978
batch 5171: loss 0.011290322057902813
batch 5172: loss 0.10453291982412338
batch 5173: loss 0.08933732658624649
batch 5174: loss 0.1524049937725067
batch 5175: loss 0.01137480791658163
batch 5176: loss 0.026317190378904343
batch 5177: loss 0.007213954348117113
batch 5178: loss 0.0627487227320

batch 5388: loss 0.01735047809779644
batch 5389: loss 0.07532758265733719
batch 5390: loss 0.04700238257646561
batch 5391: loss 0.013811538927257061
batch 5392: loss 0.014669595286250114
batch 5393: loss 0.07054777443408966
batch 5394: loss 0.020277447998523712
batch 5395: loss 0.07851995527744293
batch 5396: loss 0.06875786930322647
batch 5397: loss 0.055692870169878006
batch 5398: loss 0.035018082708120346
batch 5399: loss 0.05971524119377136
batch 5400: loss 0.015367365442216396
batch 5401: loss 0.02884652465581894
batch 5402: loss 0.021510111168026924
batch 5403: loss 0.12548957765102386
batch 5404: loss 0.008313020691275597
batch 5405: loss 0.006594356149435043
batch 5406: loss 0.07213843613862991
batch 5407: loss 0.02876076102256775
batch 5408: loss 0.023489542305469513
batch 5409: loss 0.05345015972852707
batch 5410: loss 0.03022150881588459
batch 5411: loss 0.015230881981551647
batch 5412: loss 0.05406174436211586
batch 5413: loss 0.02229466661810875
batch 5414: loss 0.03478442

batch 5623: loss 0.01691661961376667
batch 5624: loss 0.015019950456917286
batch 5625: loss 0.02949177846312523
batch 5626: loss 0.09998094290494919
batch 5627: loss 0.03722020983695984
batch 5628: loss 0.03370256349444389
batch 5629: loss 0.09690870344638824
batch 5630: loss 0.040230028331279755
batch 5631: loss 0.006415148731321096
batch 5632: loss 0.055146872997283936
batch 5633: loss 0.05692228302359581
batch 5634: loss 0.01843654178082943
batch 5635: loss 0.11376971751451492
batch 5636: loss 0.015736715868115425
batch 5637: loss 0.012142731808125973
batch 5638: loss 0.013480628840625286
batch 5639: loss 0.058514565229415894
batch 5640: loss 0.06929200887680054
batch 5641: loss 0.14117345213890076
batch 5642: loss 0.020022470504045486
batch 5643: loss 0.022107014432549477
batch 5644: loss 0.026071475818753242
batch 5645: loss 0.08602296561002731
batch 5646: loss 0.028026530519127846
batch 5647: loss 0.17650628089904785
batch 5648: loss 0.09164627641439438
batch 5649: loss 0.0086803

batch 5859: loss 0.06635650247335434
batch 5860: loss 0.044807318598032
batch 5861: loss 0.031619034707546234
batch 5862: loss 0.04561344161629677
batch 5863: loss 0.025919174775481224
batch 5864: loss 0.0485365055501461
batch 5865: loss 0.05704955756664276
batch 5866: loss 0.04164555296301842
batch 5867: loss 0.015034801326692104
batch 5868: loss 0.0717722550034523
batch 5869: loss 0.1420058161020279
batch 5870: loss 0.008207676000893116
batch 5871: loss 0.0892280861735344
batch 5872: loss 0.03698015585541725
batch 5873: loss 0.04123660549521446
batch 5874: loss 0.02604474499821663
batch 5875: loss 0.01767832785844803
batch 5876: loss 0.0747755616903305
batch 5877: loss 0.06325137615203857
batch 5878: loss 0.016910558566451073
batch 5879: loss 0.009406445547938347
batch 5880: loss 0.04169231653213501
batch 5881: loss 0.09183676540851593
batch 5882: loss 0.0049897292628884315
batch 5883: loss 0.20171630382537842
batch 5884: loss 0.04530685022473335
batch 5885: loss 0.010482053272426128

## 模型的評估 
* 使用 **tf.keras.metrics** 中的 **SparseCategoricalAccuracy** 評估器來評估模型在測試集上的性能，該評估器能夠對模型預測的結果與真實結果進行比較，並輸出預測正確的樣本數佔總樣本數的比例。

In [9]:
sparse_catgorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_catgorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print(f"test accuracy: {sparse_catgorical_accuracy.result()}")

2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step
test accuracy: 0.9725000262260437
